### imports

In [23]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import spacy
import pytextrank
import scipy as sp

### load datasets

In [64]:
songsdata_1 = pd.read_csv('lyrics1.csv')
songsdata_2 = pd.read_csv('lyrics2.csv')
songsdata_4 = pd.read_csv('lyrics4.csv')
songsdata_df = songsdata_1.append(songsdata_2)
songsdata_df = songsdata_df.append(songsdata_4)

for ix, row in songsdata_df.iterrows():
    if isinstance(row['lyrics'], float):
        songsdata_df = songsdata_df.drop(ix)

songsdata_df = songsdata_df.sample(n=2500)
        
display(songsdata_df)

,Unnamed: 0,track,artist,genre,valence_tags,arousal_tags,lyrics
514,1073,Into the Open,Heartless Bastards,rock,1.403333,1.903333,I just want to go But I didn't wanna let go I ...
670,1424,Forever Young,Rod Stewart,classic rock,5.916071,4.871071,May the good Lord be with you Down every road...
1262,2745,Dorothy At Forty,Cursive,indie rock,6.044634,5.346829,"Dorothy, I know you've had amazing dreams We c..."
607,1332,Helen,Ariel Pink's Haunted Graffiti,indie,6.327565,4.467638,"Helen, are you still there? I can't really te..."
63,144,Avalanche,Manafest,hip-hop,6.420000,5.700000,"Get me out, (out) get me out (out) PREACHER,..."
...,...,...,...,...,...,...,...
2836,6246,I Know It's True But I'm Sorry To Say,Violent Femmes,rock,1.050000,1.745000,"I know it's true, but I'm sorry to say Yesterd..."
154,344,Players of the Game,Quasimoto,hip hop,5.795000,3.800000,"""We're all apart of the game!"" Will smile in..."
2645,5830,International Lover,Prince,funk,7.412364,6.073273,"May I have this dance? Darling, it appears t..."
959,2094,Robeson,Saul Williams,spoken word,4.966667,4.291667,"*instrumental until 1:17* Beginning with, the..."


In [ ]:
def transform_data(dataset, column_ix, feature_num=500):
    features = {}
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe("textrank")

    for i, index in enumerate(dataset.index):
        if not i % 100:
            print('done: {}'.format(i))
        # print(dataset.iloc[index, column_ix])
        text = dataset.iloc[i, column_ix]
        doc = nlp(text)

        temp = []
        for phrase in doc._.phrases:
            temp.append([phrase.text, phrase.rank])
            feat = phrase.text
            rank = phrase.rank
            if feat in features:
                features[feat] += rank
            else:
                features[feat] = rank
            dataset.at[index, feat] = 1

    print(len(features))
    features_sorted = sorted([[features[x], x] for x in features.keys()], key=lambda x: x[0], reverse=True)[:feature_num]
    features_sorted = [x[1] for x in features_sorted]

    for key in features.keys():
        if key not in features_sorted:
            dataset = dataset.drop(key, axis=1, )
    return dataset.fillna(0)


x_data = transform_data(songsdata_df, 6, feature_num=400)

done: 0
done: 100
done: 200
done: 300
done: 400
done: 500
done: 600
done: 700
done: 800
done: 900
done: 1000
done: 1100
done: 1200
done: 1300
done: 1400
done: 1500
done: 1600
done: 1700
done: 1800
done: 1900
done: 2000
done: 2100
done: 2200
done: 2300
done: 2400
38749


In [63]:
display(songsdata_df)

,Unnamed: 0,track,artist,genre,valence_tags,arousal_tags,lyrics,tick,Push,Tick,...,Loves eye,his sickle hour,thy power Dost,censures,my lovely boy,fair whereon,delayd,my false eyes,doth,thy foul faults
0,1,Push Your Tush,Jessica Simpson,pop,6.216667,5.646667,Come and get it! Woo! Do ya hear that cow bel...,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Vanilla Twilight,Owl City,electronic,7.125472,4.769623,The stars lean down to kiss you And I lie awa...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,Outkast,P.O.D.,hard rock,3.025000,2.725000,I'm an outcast But don't count me out I'm an ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,Ruby Magic,Portugal. The Man,experimental,3.652857,2.805714,Someday we will find a home (I found the way ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,The Day the World Went Away (Quiet),Nine Inch Nails,industrial,3.283333,2.713333,"Na na na nah Na na na na, nah Na na na nah Na ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3408,7489,Concrete Jungle (Live),The Specials,ska,3.950000,5.570000,I'm going out tonight I don't know if I'll be ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3409,7492,Looks Like Rain,The Postmarks,indie pop,6.522087,3.941304,Lightning flashes When you glance I'm thunders...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3410,7494,Moonchild,M83,electronic,6.571648,4.587033,They say I made the Moon. Everything was in t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3411,7495,Stealing Happy Hours,311,rock,6.074438,3.874260,Massive laminate fortress Hazy cannabis porch...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
